# Data Import, Cleaning, and Preparation

This module is used to query the postgreSQL database in order to retrieve the Yelp and Violations dataset. There will be several steps to do this:
1. Import flattened violations dataset. We will use a SQL query to perform aggregation of violation data by restaurant and inspeciton date. 
2. Import Yelp business data and join to inspection data    
4. Join the Yelp business and Inspection Data with the Yelp Review Data
    + Reviews for a given establishment will be aggregated so that reviews *after* the previous inspection (or the earliest review date) and *before* the date of a given inspection are in one batch. 
    +  Aggregate any review "count" features using this same logic
    + Combine the review documents for a restaurant into a CLOB using the same logic
    
### TO DO:
1. Determine how we want to do Levenshtein matching to combine the datasets (see the *Join Yelp Review Data with Inspection Dataset* below) 
2. Create additional engineered features
3. n-gram extraction
4. vectorization of n-grams
5. model selection
    + training
    + validaiton
    + evaluation
    +repeat with additional or removed features and data segmentation (if aggregating the review text does not yield successful results)

## Import and Clean Data

In [3]:
import psycopg2 as psy
import pandas as pd
import re
import numpy as np

In [10]:
#set up connection to our DB
conn = psy.connect(database="sterndsyelp", 
                        user="mvsternds", 
                        password="nyustern123!", 
                        host="sterndsyelp.cawzspvmqd5q.us-east-1.rds.amazonaws.com", 
                        port="5432"
                       )
#open cursor and check our tables in the DB
cur = conn.cursor()

In [14]:
cur.execute("SELECT * FROM public.restaurants LIMIT 50 ")
biz = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.toronto_checkins LIMIT 50")
checkins = pd.DataFrame(cur.fetchall())

cur.execute("SELECT * FROM public.toronto_reviews LIMIT 50")
reviews = pd.DataFrame(cur.fetchall())

**NOTE: ONLY LIMITING to 50 rows during build phase to limit processing time. **

### Yelp Business Data

In [148]:
biz.columns = ['bizID', 'name', 'address', 'zip', 'neighborhood', 'lat','long', 'categories','attributes','is_open','review_count','hours','stars']
biz.describe()

,bizID,name,address,zip,neighborhood,lat,long,categories,attributes,is_open,review_count,hours,stars
count,50,50,50,50,50,50,50,50,50,50,50,50,50
unique,50,50,50,49,29,50,50,49,49,2,32,34,7
top,GY2VyNBxX5H_8kilGCAjUA,The Corktown Kitchen,323 Richmond Street E,M4E 1H7,Downtown Core,43.6548768,-79.3852158,"['Grocery', 'Food']",,1,3,,3.0
freq,1,1,1,2,5,1,1,2,2,38,6,17,15


**following section is to normalize addresses. a package is available but doing it manually seems easier / good enough. package:** https://github.com/pnpnpn/street-address 

In [149]:
#normalizes addresses
biz['address'] = [addr.replace('Street','St').replace('Boulevard','Blvd').replace('Avenue','Ave').replace('Road','Rd')
        .replace('North','N').replace('West','W').replace('South','S').replace('East','E') for addr in biz['address']]

#we should also think about removing pre and suffixes, like this example (insp data does not seem to have units etc):
biz['address'][37]

'1852 Queen St E'

### Inspection Data

In [150]:
#import toronto inspection data
cur.execute("SELECT * FROM public.toronto_inspections")
insp = pd.DataFrame(cur.fetchall())
insp.columns = ['insp_bizID','insp_biz_name','insp_biz_address','insp_date', 'last_inspection','insp_count_minor','insp_count_significant','insp_count_crucial','insp_count_na','insp_total_count_cs']
insp.head()

,insp_bizID,insp_biz_name,insp_biz_address,insp_date,last_inspection,insp_count_minor,insp_count_significant,insp_count_crucial,insp_count_na,insp_total_count_cs
0,10187087,METROPOLIS BAKESHOP,2 BLOOR ST W,3/17/17,10/18/16,0,0,0,0,0
1,10289695,MILAN'S PIZZERIA & WINGS,1792 WESTON RD,11/9/15,11/2/15,0,0,0,0,0
2,10355463,PIZZA PASTA WAYS,3300 BLOOR ST W,12/15/16,10/7/15,2,0,0,0,0
3,10453689,TRINITY - ST. PAUL'S UNITED CHURCH,427 BLOOR ST W,2/18/16,12/21/15,0,0,0,0,0
4,10503134,INTERNATIONAL NEWS SHEPPARD,45 SHEPPARD AVE E,1/31/17,1/31/17,1,0,0,0,0


### Yelp Review Data

In [151]:
#get list of unique biz and create df
uniquebiz=np.unique(biz['bizID'])
bizrevs = pd.DataFrame(uniquebiz)
bizrevs.columns = ['bizID']
bizrevs = pd.merge(bizrevs,biz[['bizID','name','address']],on='bizID', how='left')
bizrevs.head()

,bizID,name,address
0,0W0eaYgmM_PWRQjHpodBIw,ND Sushi & Grill,3 Baldwin St
1,0iTpgvz5XJzFAo253nIE0g,The Sidekick,1374 Queen St E
2,1E1BN1eJQX7j6EdepVxoMQ,Astra Meat Products,2238 Bloor St W
3,7IF5cjvH1d97pzvYPnf9Mg,Savera Indian Cuisine,815 Saint Clair Ave W
4,9TyF1tZC-wqVMU2-tZGTiA,Zane Patisserie,1852 Queen St E


In [152]:
reviews.columns = ['bizID','reviewID','userID','type','stars','text','useful','funny','cool','date']
#get dummies for star rating column
reviews = pd.concat([reviews, pd.get_dummies(reviews['stars'], prefix='stars')], axis=1)
reviews.head()

,bizID,reviewID,userID,type,stars,text,useful,funny,cool,date,stars_3,stars_4,stars_5
0,7BsdthkYwRmJpUX7huwhHA,w2sFSaC6x9OPO3Rdwk3v9w,VMFMdB6A7OofUGb1NFIuOg,review,4,Zocalo is a brand new place in the hood. Only ...,7,0,1,2010-07-02,0,1,0
1,7BsdthkYwRmJpUX7huwhHA,FRMwji3r4DXtGkihkDq9lw,a8XAdF2wOWJDp8s772GSFA,review,5,"""Damn I love this place.\nGreat drinks, food a...",2,0,0,2012-09-20,0,0,1
2,7BsdthkYwRmJpUX7huwhHA,fdBy5XzWOEyiFd7hV2Q3jA,WL0RRDIXuPMRo8khPfHVKQ,review,4,"""Run by the friendliest couple of restaurateur...",0,0,0,2011-08-02,0,1,0
3,7BsdthkYwRmJpUX7huwhHA,UGp5x1FowE9Gw423oEfOWA,YE2qAEXVuj1KOFO41Gmx4w,review,4,"""Consistently good, wholesome food! Upon enter...",0,0,0,2013-05-26,0,1,0
4,7BsdthkYwRmJpUX7huwhHA,br1HhNcCxPCihzSUbNCEZQ,mgfLGeUjMutfHmrHTr0ZLQ,review,4,"""I wanted to LOVE this place, but I ended up j...",2,0,0,2012-04-24,0,1,0


In [153]:
#declare rest of columns
bizrevs['reviews'] = ""
bizrevs['checkins'] = 0
bizrevs['stars_1'] = 0
bizrevs['stars_2'] = 0
bizrevs['stars_3'] = 0
bizrevs['stars_4'] = 0
bizrevs['stars_5'] = 0
bizrevs['reviews_whole_words'] = ""
bizrevs.head()

,bizID,name,address,reviews,checkins,stars_1,stars_2,stars_3,stars_4,stars_5,reviews_whole_words
0,0W0eaYgmM_PWRQjHpodBIw,ND Sushi & Grill,3 Baldwin St,,0,0,0,0,0,0,
1,0iTpgvz5XJzFAo253nIE0g,The Sidekick,1374 Queen St E,,0,0,0,0,0,0,
2,1E1BN1eJQX7j6EdepVxoMQ,Astra Meat Products,2238 Bloor St W,,0,0,0,0,0,0,
3,7IF5cjvH1d97pzvYPnf9Mg,Savera Indian Cuisine,815 Saint Clair Ave W,,0,0,0,0,0,0,
4,9TyF1tZC-wqVMU2-tZGTiA,Zane Patisserie,1852 Queen St E,,0,0,0,0,0,0,


In [ ]:
#look up each unique biz ID in reviews table
for i in range(len(unique(bizrevs['bizID'])):
    #add text of review to reviews column if biz IDs match
    for x in range(len(reviews['bizID'])):
        if bizrevs['bizID'][i] == reviews['bizID'][x]:
            bizrevs['reviews'][i] = bizrevs['reviews'][i] + reviews['text'][x]
    #count number of checkins
    for y in range(len(checkins['bizID'])):
        if bizrevs['bizID'][i] == checkins['bizID'][y]:
            bizrevs['checkins'][i] = bizrevs['checkins'][i] + 1
    #count number of reviews with each star rating
    for z in range(len(reviews['bizID'])):
        if bizrevs['bizID'][i] == reviews['bizID'][z]:
            bizrevs['stars_1'][i] = bizrevs['stars_1'][i] + reviews['stars_1'][z]
            bizrevs['stars_2'][i] = bizrevs['stars_2'][i] + reviews['stars_2'][z]
            bizrevs['stars_3'][i] = bizrevs['stars_3'][i] + reviews['stars_3'][z]
            bizrevs['stars_4'][i] = bizrevs['stars_4'][i] + reviews['stars_4'][z]
            bizrevs['stars_5'][i] = bizrevs['stars_5'][i] + reviews['stars_5'][z]
    #extract whole words from reviews
    bizrevs['reviews_whole_words'][i] = ' '.join(re.findall('[A-Za-z]+', bizrevs['reviews'][i]))
    
bizrevs.head()

### Join Yelp Review Data with Inspection Dataset

We have a few options here. While it is optimal to have as much done in Python as possible, the matching process in python is impractically slow. We can 1) use a manual implementation of Levenshtein Distance (LD), 2) use a package with optimized LD code, or 3) do the joining of the inspeciton data and review data in our database. OPtions 1& 2 are shown below, and code to retrieve the results of option 3 are at the bottom of this section.

If we decide to go with option 3, the last steps are to combine all records where the business ID, last inspection date, and inspection date are equal in order to get to one observation per restaurant-inspection combination. We can then add in any other engineered features.

**note: next cell should return matches once we include more than the 50 rows (fingers crossed)**

#### Levenshtein Option #1 (package)

In [135]:
import pylev as pl
#loop through each yelp bizID and find the restaurant with closest lev distance (currently matches using name only)
#left join the inspection data to the yelp business table so that the reviews can be aggregated on bizID and inspection 
biz['lev_dist'] = ""
biz['insp_bizID'] = ""
for i in range(len(biz['bizID'])):
    for x in range(len(insp['insp_bizID'])):
        dist = pl.levenshtein(str(biz['name'][i]),str(insp['insp_biz_name'][x]))
        if dist < 5:
            biz['lev_dist'][i] = dist
            biz['insp_bizID'][i] = insp['insp_bizID'][x]
df = pd.merge(biz,insp,on='insp_bizID', how='left')            
df.head()

,bizID,name,address,zip,neighborhood,lat,long,categories,attributes,is_open,...,insp_bizID,insp_biz_name,insp_biz_address,insp_date,last_inspection,insp_count_minor,insp_count_significant,insp_count_crucial,insp_count_na,insp_total_count_cs
0,vhkBFO4vEMwmTpX_1ooGVg,Future Bakery & Cafe,"95 Front St E, St. Lawrence Market",M5E 1C2,Corktown,43.6493564,-79.3714125,"['Bakeries', 'Food']","['BikeParking: True', 'BusinessAcceptsCreditCa...",1,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fRMLGUfKz5tl166C63lvoQ,"""Freddie's No Frills""",243 Alberta Ave,M6C 3X4,,43.6811055,-79.4338303,"['Grocery', 'Food']","['BikeParking: True', 'BusinessAcceptsCreditCa...",1,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6BC87j5FxoIwa-atC77WYQ,Tab\xc3\xbcl\xc3\xa8 Middle Eastern Cuisine,810 Queen St E,M4M 1H7,Riverdale,43.659652,-79.346481,"['Restaurants', 'Middle Eastern']","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kn6bzO3-KZhsxSExDUsOCA,Thai Spicy House,517 Mount Pleasant Rd,M4S 2M4,Mount Pleasant and Davisville,43.7019747414,-79.3873448973,"['Thai', 'Restaurants']","['Alcohol: full_bar', ""Ambience: {'romantic': ...",1,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,vvA99tqHFMVgHnB7WmhO1w,Sandwich Box,37 Eglinton Ave E,M4P 1A6,Mount Pleasant and Davisville,43.7067926,-79.3968422,"['Restaurants', 'Sandwiches']","['Alcohol: none', ""Ambience: {'romantic': Fals...",0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Levenshtein Option #2 (manual)

In [ ]:
#declare function to calculate levenshtein distance between 2 strings (not case sensitive)
def lev(string1, string2):
    #delete the ".lower()" in the following two lines to make distance case sensitive
    s1=str.strip(string1.lower())
    s2=str.strip(string2.lower())
    m=len(s1)+1
    n=len(s2)+1

    tbl = {}
    for i in range(m): tbl[i,0]=i
    for j in range(n): tbl[0,j]=j
    for i in range(1, m):
        for j in range(1, n):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            tbl[i,j] = min(tbl[i, j-1]+1, tbl[i-1, j]+1, tbl[i-1, j-1]+cost)

    return tbl[i,j]

#test the function
print(lev('Hello',"hello"))
print(lev('dock','duck '))
print(lev('st','saint'))

In [118]:
#set value of levenshtein distance threshold (4 means only distances of 3 and lower would be considered)
lev_dist_threshold = 4

#loop through each yelp bizID and find the restaurant with closest lev distance (currently matches using name only)
bizrevs['lev_dist'] = lev_dist_threshold
bizrevs['insp_bizID'] = ""
for i in range(len(bizrevs['bizID'])):
    for x in range(len(insp['insp_bizID'])):
        dist = lev(str(bizrevs['name'][i]),str(insp['insp_biz_name'][x]))
        if dist < bizrevs['lev_dist'][i]:
            bizrevs['lev_dist'][i] = dist
            bizrevs['insp_bizID'][i] = insp['insp_bizID'][x]

df = pd.merge(bizrevs,insp,on='insp_bizID', how='left')            

df.head()

/Users/cmvoelker/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/cmvoelker/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


KeyboardInterrupt: 

**Note: merge in cell above won't result in matches for name/address because of 50 row limit put in place. should work after removing limit**

#### Levenshtein Option #3 (in-database) 
This option joins the yelp restaurant informaiton to each inspection record where:
 * The Levenshetein distance of the restaurant name from the two datasets is <3
 * The distance of the address from each dataset is <4
 * The date of the review is greater than the prior inspection date
 * The date of the review is less than or equal to inspeciton date on the record
 
Whitespace at the beginning and end of the name and address in each dataset is trimmed, and the strings are converted to uppercase before matching. The mathcing thresholds can be adjusted to increase potential for matching, or decrease false matches.

In [11]:
# The materialized view of the restaurant, inspection, and review data is "toronto_all"
cur.execute("SELECT * FROM public.toronto_all where review_date is not null limit 250" )
obs = pd.DataFrame(cur.fetchall())
obs.head()


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,OriPsMQx1cRyE9hgNnhQFg,m7z-tX6XDZ27xGhGjnI21w,5,"""I am confident when I say this place is hands...",4,0,4,2016-02-15
1,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,1WON5dUarKdWJ6-yq7Ne6g,UrfdzamoBt0WW9Ifqy7RIw,5,"The short version of my review is this: ""Hashi...",3,0,1,2015-11-18
2,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,TNpm4Hs6x2yJvYbUZSeA4w,5KbUkX5DHGtDSmqdG5LLhw,5,"""It's a great cultural experience with great f...",0,0,0,2016-03-14
3,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,0pS7e898Z5AywcZQ9_DX1Q,xspGyCnzmZgsP-VUnQ3K4A,5,"""This was definitely where quality of food and...",1,0,1,2016-03-15
4,01l8MH9tBK6GPvJQdMU1gw,Kaiseki Yu-zen Hashimoto,6 Garamond Court,M3C 1Z5,,43.7264546,-79.3349744,"['Restaurants', 'Japanese']","['Alcohol: beer_and_wine', ""Ambience: {'romant...",1,...,0,0,ePOLGcC5yJbD_DzOLfyKbA,_dRcIdWjks0phgfiY27AFQ,5,My sister and I enjoyed the lunchtime Kaiseki ...,0,0,0,2016-05-01


### Checkin Data

In [93]:
checkins.columns = ['bizID','type','datetime']
checkins.describe()

,bizID,type,datetime
count,50,50,50
unique,50,1,50
top,mhm5282-LI8Ddq3txkijYQ,b'checkin',"['Mon-0:1', 'Sun-0:1', 'Thu-0:2', 'Wed-0:1', '..."
freq,1,50,1
